**TP n°5** : Learning to Rank et Ré-identification

#Plan

## Partie I: un problème de learning-to-rank

- Learning to order things 
- RankNet loss
- Listwise Loss

## Partie II: un problème de ré-identification

- triplet loss avec MNIST + triplet loss
- plus dur avec données webcam

Durée : 4 h

**Partie I** :

On distingue plusieurs problème sous l'étiquette "[learning-to-rank](https://link.springer.com/content/pdf/10.1007/978-3-642-15880-3_20.pdf)".\
Il peut s'agir, par exemple de ranger une liste de contenus par pertinence en fonction d'une requête. Ici, le tri s'effectue sur les possibles sorties du modèle : on parle de *label ranking*.\
Il peut aussi s'agir de trier des images dans un ensemble en fonction d'un critère particulier.
Là, le tri s'effectue sur les entrées du modèle; pour décrire cette situation, les termes suivants sont souvent utilisés: *object ranking*, *learning to order things*. \
Mais dans les deux cas, on ne dispose pour apprendre que d'arrangements. Par exemple des paires d'images ordonnées. 

Dans ce TP, nous illustrons la deuxième situation, à partir d'images de synthèse très simples. Toutes les images contiennent en mélange un disque et un nombre variable de rectangles de formes différentes. Le but est de trier les images en fonction de l'intensité des pixels sur le disque.\
Pour le faire, nous nous plaçons dans un contexte standard où on dispose de paires d'images ordonnées. Sur ces paires, nous entraînerons un réseau de neurones pour construire une "fonction de rang" (*ranker*) à valeurs réelles dont les sorties permettent de trier les images.


**Exercice n°1** : construction du problème

Les cellules suivantes permettent de:
- générer un jeu de données sur votre drive (train+val et test)
- définir un Dataset qui met à disposition des paires d'images et une comparaison sur le critère de l'intensité du disque ("0" si le disque est plus intense sur la première image, "1" sinon).
- visualiser un premier batch

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/MyDrive/...')

MessageError: ignored

In [ ]:
import os
from os.path import join
ls = lambda rep: sorted(os.listdir(rep))

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader, sampler

import matplotlib.pyplot as plt
import copy
from random import randint, choice

#other
#from datasets import *
from archis import *
from utile_tp5I import *
#from train_and_test import *
 
root = r"."

In [ ]:
dir_trainval = join(root, r"train")
generate_dataset(dir_trainval, size_dataset=10000)

dir_test = join(root, r"test")
generate_dataset(dir_test, size_dataset=2000)

In [ ]:
# Récupérations des valeurs cibles (normalement inaccessibles)

# path des images:
dir_images_trainval = os.path.join(dir_trainval, 'images')
dir_images_test = os.path.join(dir_test, 'images')

# valeurs cibles train+val
name_dic = os.path.join(dir_trainval, 'labels_synthese.pickle')
with open(name_dic, 'rb') as handle:
    dic = pickle.load( handle)


# valeurs cibles test
name_dic_test = os.path.join(dir_test, 'labels_synthese.pickle')
with open(name_dic_test, 'rb') as handle:
    dic_test = pickle.load( handle)
    
# split train / val (8000/2000)
names = np.array(ls(dir_images_trainval))

train_indices = list(range(0,8000))
names_train = names[train_indices]
val_indices = list(range(8000,10000))
names_val = names[val_indices]
names_test = ls(dir_images_test)

# nb: pour sep. aléatoire : utiliser sklearn.model_selection.train_test_split as tts

In [ ]:
# si erreur :
# from shutil import rmtree
# rmtree(dir_test)

In [ ]:
#%% Augmentation de données
class super_flip(object):
    """
    Les 8 transformations
    générées par R(Pi/2) et sym/axe vertical
    """
    def __init__(self,nb):
        self.nb=nb
    def __call__(self, image):
        # hérésie : normalement, torch.randint...
        n = randint(0, self.nb)
        if n==1:
            image= image.flip([1])
        elif n==2:
            image = image.flip([2])
        elif n==3:
            image = image.transpose(1,2)
        elif n==4:
            image = image.transpose(1,2).flip([1])
        elif n==5:
            image = image.transpose(1,2).flip([2])
        elif n==6:
            image = image.flip([1,2])
        elif n==7:
            image = image.transpose(1,2).flip([1,2])
        return image

Sflip = super_flip(8)

tr = {
    'train': Sflip,
    'val': None,
    'test': None
}

In [ ]:
# Construction des datasets:
def oracle(name0, name1, dic):
    #load the data:
    y0 = dic[name0]['y']
    y1 = dic[name1]['y']
    
    #get the compa:
    if y1 < y0:
        compa = 0
    else:
        compa = 1
    return compa


class Dataset_ordered_pairs(torch.utils.data.Dataset):
    def __init__(self, images_dir,  dic, transfo = None):
        self.images_dir = images_dir
        self.transfo = transfo
        self.imgs = ls(images_dir)
        self.dic = dic 

    def __getitem__(self,idx):
                
        name0 = self.imgs[idx]
        name1 = choice(self.imgs)   
        label = oracle(name0, name1, self.dic)
            
        #get the images
        path0 = os.path.join(self.images_dir, name0)
        img0 =  torch.load(path0)
        path1 = os.path.join(self.images_dir, name1)      
        img1 = torch.load(path1)

        if self.transfo is not None:
            img0 = self.transfo(img0)                
            img1 = self.transfo(img1)

        return img0, img1, torch.from_numpy(np.array(label)).long(), name0, name1   #-1 si pas de classe 0

    def __len__(self):
        return len(self.imgs) 

In [ ]:
dataset_train = Dataset_ordered_pairs(dir_images_trainval, dic, tr['train'])
dataset_val = Dataset_ordered_pairs(dir_images_trainval, dic, tr['val'])
dataset_test = Dataset_ordered_pairs(dir_images_test, dic_test, tr['test'])

ds = {'train' : dataset_train , 'val' :dataset_val, 'test':dataset_test }


In [ ]:
# Samplers et loaders
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)

samplers={'train':train_sampler,'val':val_sampler}

batch_size = 32

dataloaders = {x: torch.utils.data.DataLoader(ds[x], batch_size=batch_size, shuffle=False, sampler = samplers[x], num_workers=0) for x in ['train', 'val']}             
dataloaders['test'] = torch.utils.data.DataLoader(ds['test'], batch_size=batch_size, shuffle=False, num_workers=0)
dataset_sizes = {'train': len(names_train), 'val': len(names_val), 'test': len(names_test)}

dataloaders['viz'] = torch.utils.data.DataLoader(ds['train'], batch_size=6, shuffle=False, num_workers=0)

In [ ]:
# Visualisation

img1, img2, labels, _, _ = next(iter(dataloaders['test']))

fig0 = plt.figure(0, figsize=(16, 2))
voir_batch2D(img1, nx = 8, fig = fig0, k=0, min_scale=0,max_scale=10) 
fig1 = plt.figure(1, figsize=(16, 2))
voir_batch2D(img2, nx = 8, fig = fig1, k=0, min_scale=0,max_scale=10) 

print(labels)


**Q0** Comment sépare-t-on entraînement et validation ici ? 

**Q1** Quel est le rôle de *superf_flip*? Celui de la fonction *oracle* ?

**Q2** Les paires d'images sont-elles toutes aussi faciles à ordonner ? 

**Exercice n°2** : apprentissage en **siamois**

Au cours d'un entraînement, on génère des (batches de) paires d'images comparées. L'entraînement de réseaux **siamois** basique consiste à passer le modèle sur chaque image de la paire indépendemment, puis à pénaliser le modèle lorsque les sorties sont arrangées dans le mauvais ordre.

Le plus simple, pour le faire, est de considérer la partie positive de la différence entre les sortie. C'est ce que fait la fonction de coût suivante:

In [ ]:
# fonction Hinge Loss

def label_to_sgn(label):  #0 -> 1  et 1 -> -1 
    sgn =copy.deepcopy(label)
    sgn.detach()
    sgn[label==0] = 1
    sgn[label==1] = -1
    return sgn



class HingeLoss(torch.nn.Module):
    def __init__(self, margin = 0.1):
        super(HingeLoss, self).__init__()
        self.margin = margin
        
    def forward(self, output0, output1, label):
        sgn = label_to_sgn(label)
        diff = sgn*(output1 - output0)
        
        loss = torch.relu(diff + self.margin).mean()
        return loss


**Q0** Que se passe-t-il si on a des sorties $y_0 > y_1$ alors que le disque de l'image 1 est plus intense que celui de l'image 0 ?\
Dans quels autres cas la fonction de coût est-elle strictement positive ?

**Q1** Ecrire la boucle d'apprentissage et lancer sur 20 époques. Garder les justesses successives en mémoire.

In [ ]:
num_epochs = 20
channels = 1 

model = vgg11(pretrained=False, progress=True, channels=channels, num_classes=1, init_weights=True)
device = torch.device("cuda:0")
model = model.to(device) 
    
    
criterion = HingeLoss(0.1)
optimizer = optim.Adam(model.parameters(), lr = 0.001 )

In [ ]:
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
train_accs = []
val_accs = []


phases = ['train', 'val']

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in phases:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for img1, img2, labels, _ , _ in dataloaders[phase]:
            img1 = 
            img2 = 
            #print(inputs)
            labels = labels.to(device).detach()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                output1 =
                output2 = 
                preds = 
#                    loss = torch.mean(output1 - output2)  
                loss =  criterion( )

                # backward + optimize only if in training phase
                if phase == 'train':
                    #pass
                    loss.backward()
                    optimizer.step()
           
            
            # statistics
            running_loss += loss.item() * img1.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            #del
            del img1
            del img2
            del labels
            del loss
            del output1
            del output2
            torch.cuda.empty_cache()
            
        

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        if phase == 'train':
          train_accs.append(epoch_acc)

        if phase == 'val':
          val_accs.append(epoch_acc)

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())



In [ ]:
# load best model weights
# model.load_state_dict(best_model_wts)

**Q3** Visualiser les résultats. La justesse vous paraît-elle une bonne mesure des performances ?

**Q4** Comment amélioreriez-vous les performances ?





**Exercice n°3** RankNet Loss (et ListNet Loss)

Une version plus douce de la Hinge Loss a été très utilisée, en particulier pour l'apprentissage de moteurs de recherche. Il s'agit de la RankNet Loss.

Cette fonction de coût est dérivée d'un modèle probabiliste paramétrique, [le modèle de Bradley-Terry](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model).

Dans une version générale, on suppose que le résultat d'une comparaison (ou d'un match) entre deux objets "0" et "1" (ou deux équipes) est aléatoire, et dépend de réels associés aux objets (les "niveaux des équipes") suivant:
\begin{align}
P_0 = \dfrac{f(y_0)}{f(y_0) + f(y_1)}
\tag{1}
\end{align}
Où $P_0$ est la probabilité de choisir l'objet "0" (ou que la première équipe gagne) et $f$ est une fonction strictement croissante à valeurs positives.

**Q1** Dans le cas où $f(y) = e^{\sigma y}$, de quoi dépendent les probabilités de choix ?
Ecrire la log-vraisemblance de l'événement "l'objet $x$ est choisi". 

**Q2** En déduire une fonction de coût adaptée à notre problème de ranking.

**Q3** Implémenter et comparer sur vingt époques.

**Q4** On peut généraliser l'équation (1). Il ne s'agit plus de préférer un objet parmi deux, mais d'ordonner $N$ objets.
En déduire une fonction de coût. L' implémenter.

https://cran.rstudio.com/web/packages/PlackettLuce/vignettes/Overview.html